In [3]:
import os

current_working_directory = os.getcwd()

print("Current working directory:", current_working_directory)

Current working directory: d:\Python\StockToolsPro\AutoGluon\TrainModel


查看torch 是否成功安裝，然後查看GPU是否可用

In [4]:
import torch

# Check if torch is installed
print("Torch installed version:", torch.__version__)

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")


Torch installed version: 2.0.1
GPU is available


顯卡資訊

In [5]:
!nvidia-smi

Sun Apr 14 01:47:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 3000              WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8              6W /   80W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

針對美股訓練數據中的Signal欄位做訓練
y --> Signal欄位

In [9]:
import pandas as pd
import os
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

# 步驟1: 合併CSV檔案
def combine_csv_files(folder_path):
    combined_df = pd.DataFrame()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            temp_df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
    combined_df = combined_df.dropna()  # 刪除缺失值
    return combined_df

# 步驟3: 分割資料集
def split_data(df):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    return train_df, test_df

# 步驟4: 使用AutoGluon訓練模型並啟用GPU加速
def train_model(train_df, label_column='Signal'):
    model_path = r"D:\Temp\StockData\model\AutoGluon\SwingTradeSignals\SwingTrade-1"
    predictor = TabularPredictor(label=label_column, path=model_path).fit(train_data=train_df, presets='best_quality', num_gpus=1 ,
                                                                          hyperparameters='extreme') # 為整個Tabular Predictor指定1個GPU
    return predictor

# 步驟5: 評估模型
def evaluate_model(predictor, test_df):
    predictor.leaderboard(test_df)  # 顯示模型的領導者榜
    performance = predictor.evaluate(test_df)  # 評估模型的性能
    best_model = predictor.model_best  # 顯示最佳模型
    print(performance)
    print(best_model)

# 主程式
if __name__ == "__main__":
    folder_path = r"D:\Temp\StockData\US_STOCK_DATA\TargetData\subfolder_1"  # 修改為你的資料夾路徑
    combined_df = combine_csv_files(folder_path)
    train_df, test_df = split_data(combined_df)
    predictor = train_model(train_df, label_column='Signal')
    evaluate_model(predictor, test_df)


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: D:\Temp\StockData\model\AutoGluon\SwingTradeSignals\SwingTrade-1/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "D:\Temp\StockData\model\AutoGluon\SwingTradeSignals\SwingTrade-1/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:  

[1000]	valid_set's binary_error: 0.34646
[2000]	valid_set's binary_error: 0.307737
[3000]	valid_set's binary_error: 0.289453
[4000]	valid_set's binary_error: 0.274197
[5000]	valid_set's binary_error: 0.267007
[6000]	valid_set's binary_error: 0.257007
[7000]	valid_set's binary_error: 0.251058
[8000]	valid_set's binary_error: 0.247774
[9000]	valid_set's binary_error: 0.245365
[10000]	valid_set's binary_error: 0.242117
[1000]	valid_set's binary_error: 0.346071
[2000]	valid_set's binary_error: 0.308916
[3000]	valid_set's binary_error: 0.289609
[4000]	valid_set's binary_error: 0.276944
[5000]	valid_set's binary_error: 0.268805
[6000]	valid_set's binary_error: 0.26125
[7000]	valid_set's binary_error: 0.255703
[8000]	valid_set's binary_error: 0.252527
[9000]	valid_set's binary_error: 0.248513
[10000]	valid_set's binary_error: 0.245556
[1000]	valid_set's binary_error: 0.350341
[2000]	valid_set's binary_error: 0.313515
[3000]	valid_set's binary_error: 0.292857
[4000]	valid_set's binary_error: 0

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
from autogluon.tabular import TabularPredictor


# 預測並輸出結果到CSV
def predict_and_export(predictor, data_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(data_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(data_folder, file_name)
            data_df = pd.read_csv(file_path)
            predictions = predictor.predict(data_df)
            data_df['Predicted_Signal'] = predictions
            output_path = os.path.join(output_folder, file_name)
            data_df.to_csv(output_path, index=False)
    print("All predictions exported successfully.")


# 主程式
if __name__ == "__main__":
    data_folder = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"  # 修改為你的資料夾路徑
    output_folder = r"D:\Temp\StockData\TW_STOCK_DATA\Output"  # 設定輸出資料夾
    model_path = r"D:\Temp\StockData\model\AutoGluon\SwingTradeSignals\SwingTrade-1"
    predictor = TabularPredictor.load(model_path)  # 加載模型
    predict_and_export(predictor, data_folder, output_folder)
